**1**. (100 points)

Write a predicate function `is_prime` that efficiently checks whether a number is prime. Use this to write a second function `primes_between` that returns the prime numbers between two integers as a `numpy` array.

- (10 points) Do this in regular Python 
- (10 points) Accelerate using `numba` (serial version) 
- (15 points) Accelerate using `numba` (parallel version)
- (10 points) Accelerate using `cython` (serial version) 
- (15 points) Accelerate using `cython` (parallel version)
- (10 points) Report the speed-up multiplier as an integer of the `numba` and `cython` serial and parallel versions using `timeit` in a DataFrame for the numbers between 0 and 1,000,000
- (10 points each) Run the serial version of the python `primes_between` function in parallel using
    - `multiprocessing`
    - `joblib`
    - `ipyparallel`

- (10 points) Do this in regular Python 



In [184]:
import numpy as np
import cython
import timeit
import numba
import multiprocessing as mp
import pandas as pd

from numba import jit, njit, boolean, int32, int64
from multiprocessing import Pool, Value
from joblib import Parallel, delayed
from ipyparallel import Client, parallel, require

In [160]:
def is_prime(num):
    '''Check if the number is a prime.'''
    import numpy as np
    if num <= 1:
        return False
    for i in range(2, int(np.sqrt(num)) + 1):
        if(num % i == 0):
            return False
    return True

def primes_between(x, y):
    '''Find all the prime numbers between x and y.'''
    
    idx = list(map(is_prime, range(x, y)))
    return np.arange(x, y)[idx]

In [102]:
%timeit primes_between(0, int(1e6))

6.49 s ± 43.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- (10 points) Accelerate using `numba` (serial version) 



In [56]:
@numba.vectorize(["boolean(int32)", "boolean(int64)"])
def is_prime_numba(num):
    '''Check if the number is a prime.'''
    
    if num <= 1:
        return False
    for i in range(2, int(np.sqrt(num)) + 1):
        if(num % i == 0):
            return False
    return True

@njit
def primes_between_numba(x, y):
    '''Find all the prime numbers between x and y.'''
    
    idx = is_prime_numba(np.arange(x, y))
    return np.arange(x, y)[idx]

In [169]:
%timeit primes_between_numba(0, int(1e6))

310 ms ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- (15 points) Accelerate using `numba` (parallel version)



In [62]:
@numba.vectorize(["boolean(int32)", "boolean(int64)"], target = 'parallel')
def is_prime_numba_parallel(num):
    '''Check if the number is a prime.'''
    
    if num <= 1:
        return False
    for i in range(2, int(np.sqrt(num)) + 1):
        if(num % i == 0):
            return False
    return True

def primes_between_numba_parallel(x, y):
    '''Find all the prime numbers between x and y.'''
    
    idx = is_prime_numba_parallel(np.arange(x, y))
    return np.arange(x, y)[idx]

In [63]:
%timeit primes_between_numba_parallel(0, int(1e6))

143 ms ± 4.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


- (10 points) Accelerate using `cython` (serial version) 



In [2]:
%load_ext cython

In [81]:
%%cython -a

import cython
import numpy as np
cimport numpy as np
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def is_prime_cython(int num):
    '''Check if the number is a prime.'''
    
    cdef int i
    cdef int k
    k = int(sqrt(num))
    
    if num <= 1:
        return False
    for i in range(2, k + 1):
        if(num % i == 0):
            return False
    return True

def primes_between_cython(x, y):
    '''Find all the prime numbers between x and y.'''
    
    idx = list(map(is_prime_cython, range(x, y)))
    return np.arange(x, y)[idx]

In [170]:
%timeit primes_between_cython(0, int(1e6))

290 ms ± 9.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- (15 points) Accelerate using `cython` (parallel version)



In [98]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force -I /usr/local/opt/libomp/include -L /usr/local/opt/libomp/lib

import cython
import numpy as np
cimport numpy as np
from libc.math cimport sqrt
from libcpp cimport bool
from libc.string cimport memset
from cython.parallel import parallel, prange

@cython.boundscheck(False)
@cython.wraparound(False)
cdef bool is_prime_cython_parallel(int num) nogil except +:
    '''Check if the number is a prime.'''
    
    cdef int i
    cdef int k
    k = int(sqrt(num))
    
    if num <= 1:
        return False
    for i in range(2, k + 1):
        if(num % i == 0):
            return False
    return True

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def primes_between_cython_parallel(int x, int y):
    '''Find all the prime numbers between x and y.'''
        
    cdef int i
    cdef bool[:] idx
    idx = np.zeros((y - x), dtype = "bool")
    
    with cython.nogil, parallel():
        for i in prange(x, y):
            idx[i] = is_prime_cython_parallel(i)
    
    return np.arange(x, y)[idx]

In [100]:
# don't know why this does not speed up
%timeit primes_between_cython_parallel(0, int(1e6))

1min 24s ± 5.97 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


- (10 points) Report the speed-up multiplier as an integer of the `numba` and `cython` serial and parallel versions using `timeit` in a DataFrame for the numbers between 0 and 1,000,000



In [176]:
def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)
    return wrapped

wrapped = wrapper(primes_between, 0, int(1e6))
origin = timeit.timeit(wrapped, number = 10)/10

In [177]:
origin

6.175795529999959

In [178]:
wrapped = wrapper(primes_between_numba, 0, int(1e6))
na = timeit.timeit(wrapped, number = 10)/10

wrapped = wrapper(primes_between_numba_parallel, 0, int(1e6))
nap = timeit.timeit(wrapped, number = 10)/10

wrapped = wrapper(primes_between_cython, 0, int(1e6))
cy = timeit.timeit(wrapped, number = 10)/10

# need too much time
#wrapped = wrapper(primes_between_cython_parallel, 0, int(1e6))
#cyp = timeit.timeit(wrapped, number = 10)/10
cyp = 84.0

In [190]:
speed_up = origin / np.array([na, nap, cy, cyp])
speed_up = np.array(speed_up, dtype = 'int64')
df = pd.DataFrame(dict(func = ['numba_serial', 'numba_parallel', 'cython_serial', 'cython_parallel'],
                       speed_up = speed_up))
df

,func,speed_up
0,numba_serial,21
1,numba_parallel,44
2,cython_serial,22
3,cython_parallel,0


- (10 points) `multiprocessing`



In [126]:
xs = list(range(0, int(1e6), int(1e6) // 10))
ys = xs[1:] + [int(1e6)]

In [127]:
%%time

with mp.Pool(processes = 4) as pool:
    primes = pool.starmap(primes_between, [(x, y) for x, y in zip(xs, ys)])

np.concatenate(primes)

CPU times: user 15.6 ms, sys: 141 ms, total: 156 ms
Wall time: 3.35 s


array([     2,      3,      5, ..., 999961, 999979, 999983])

- (10 points) `joblib`



In [129]:
%%time

res = Parallel(n_jobs = 4)(delayed(primes_between)(x, y) for x, y in zip(xs, ys))

np.concatenate(res)

CPU times: user 31.2 ms, sys: 31.2 ms, total: 62.5 ms
Wall time: 3.23 s


array([     2,      3,      5, ..., 999961, 999979, 999983])

- (10 points) `ipyparallel`

In [164]:
%%time

rc = Client()
dv = rc[:]

@dv.parallel(block = True)
@require(is_prime)
def primes_between_ipy(x, y):
    '''Find all the prime numbers between x and y.'''
    import numpy as np
    
    idx = list(map(is_prime, range(x, y)))
    return np.arange(x, y)[idx]

np.concatenate(primes_between_ipy.map(xs, ys))

array([     2,      3,      5, ..., 999961, 999979, 999983])

CPU times: user 188 ms, sys: 46.9 ms, total: 234 ms
Wall time: 3.21 s
